*RunnableWithMessageHistory*  

를 활용한 메모리 기능 구현
RunnableWithMessageHistory 클라스의 등장으로 매우 쉬워졌다.

In [ ]:
# 랭체인에서 제공하는 대화방식의 프롬프트 템플릿 및 메시지 플레이스홀더 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder     

# 랭체인에서 제공하는 메모리 기반 채팅 메시지 히스토리 임포트
from langchain_core.chat_history import InMemoryChatMessageHistory

# 랭체인에서 제공하는 메시지 히스토리 기반 실행기 임포트
from langchain_core.runnables.history import RunnableWithMessageHistory  

from langchain_google_genai import ChatGoogleGenerativeAI     # 랭체인에서 제공하는 구글 생성형 AI 래퍼
from langchain_core.output_parsers import StrOutputParser     # 랭체인에서 제공하는 문자열 출력 파서

from dotenv import load_dotenv                                # .env 파일에서 환경변수를 로드하는 함수
load_dotenv()                                                 # .env 파일에서 환경변수를 로드


In [ ]:
# ai 모델을 만든다
llm_model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=1.0)

In [ ]:
# 프롬프트 설정을 한다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 아인쉬타인에 버금가는 훌륭한 물리학자야"),             # 시스템 메시지
    MessagesPlaceholder(variable_name="history"),                    # 대화 히스토리 플레이스홀더
    ("human", "{question}")                                          # 유저 메시지
])


In [ ]:
# chain 실행기를 만든다
chain = prompt | llm_model  | StrOutputParser()

In [ ]:
# 대화 히스토리를 저장할(history에 너을) 메모리 객체를 만든다

store = {}     # 대화 히스토리를 저장할 딕셔너리 객체

def get_message_history(session_id: str):
    """세션 ID에 해당하는 메모리 기반 대화 히스토리 객체를 반환하는 함수"""
    
    # 현재 대화가 저장 되어 있는지 확인
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
            
    return store[session_id]

In [ ]:
# chain 을 메모리 기반 실행기로 래핑한다

with_memory_chain = RunnableWithMessageHistory(chain,                               # 실행할 체인
                                               get_message_history,                 # 대화 기록을 가져오는 함수
                                               input_messages_key="question",       # 사용자의 질문
                                               history_messages_key="history")     # prompt 에서 어느 변수에다 넣을찌 

In [ ]:
# 메모리 기반 실행 테스트

# 첫번째 질문
response1 = with_memory_chain.invoke({"question": "내 이름은 김철수야 잘 부탁해!"},
                                      config={"configurable" : {"session_id" : "chulsu"}})

print(response1)   # 첫번째 질문을 출력한다
print("------------------------------------------------------")

# 두번째 질문
response2 = with_memory_chain.invoke({"question": "내 이름이 뭔지 기억 해?"},
                                      config={"configurable" : {"session_id" : "chulsu"}})

print(response2)   # 두번째 질문을 출력한다

In [ ]:
print(store["chulsu"].messages)   # 대화 히스토리가 저장된 store 객체를 출력한다